In [38]:
def activation_function(x,s=1):
    return 1/(1+math.exp(-s*x))

def activation_func_derivate(f,s=1):
    return s*f*(1-f)

def mean_absolute_error(pair_list):
    sum_so_far = 0.0
    for pair in pair_list:
        sum_so_far += abs(pair[1] - pair[0])
    return sum_so_far

def sum_squared_error(pair_list):
    sum_so_far = 0.0
    for pair in pair_list:
        sum_so_far += (pair[1] - pair[0])**2
    return sum_so_far

def mean_square_error(pair_list):
    size = len(pair_list)
    sum_so_far = 0.0
    for pair in pair_list:
        sum_so_far += (pair[1] - pair[0])**2
    return sum_so_far/size



def test_case(inputs,weights,bias,output_matrix):
    
    for i in range(0,len(weights)):
        for j in range(0,len(weights[i])):
            output = 0
            for k in range (0,len(weights[i][j])):
                if i == 0:
                    output += (weights[i][j][k]*inputs[k])
                else:
                    output += weights[i][j][k]*output_matrix[i-1][k]
            output += bias[i][j]
            output_matrix[i][j] = activation_function(output)
            
    return output_matrix[i][j]

In [39]:
import pandas as pd
import random
import math

def back_propagation_single_output(dataset_path,neurons_per_hidden_layer,max_iterations,split_ratio=0.8,learning_rate=0.5):
    df = pd.read_csv(dataset_path) 
    
    number_of_attributes = list(df.shape)[1]

    number_of_records = list(df.shape)[0]

    list_of_records = df.to_numpy().tolist()
        
    neurons_per_layer = [number_of_attributes-1]
    
    for neuron in neurons_per_hidden_layer:
        neurons_per_layer.append(neuron)
        
    neurons_per_layer.append(1)

    number_of_layers = len(neurons_per_layer) ##total layers in the NN
    
    ##train test split
    train_set = []
    test_set = []

    train_set_length = round(split_ratio*number_of_records,0)
    test_set_length = number_of_records-train_set_length

    for row in list_of_records:
        test_or_train = round(random.uniform(0,1),1)
        if len(train_set) == train_set_length:
            test_set.append(row)
        elif len(test_set) == test_set_length:
            train_set.append(row)
        else:    
            if test_or_train < split_ratio:
                train_set.append(row)
            else:
                test_set.append(row)
                
    ##extracting data for training
    input_matrix = [row[:-1] for row in train_set]
    target_list = [row[-1] for row in train_set]

    
    ##initializing the weight,bias and change in weight,change in bias matrices
    weight_matrix = []

    bias_matrix = []

    change_in_weight_matrix = []

    change_in_bias_matrix = []
    #first row contains input layer,last row contains one layer before output layer
    for i in range(0,number_of_layers-1): 
        weight_row = []
        bias_row = []
        dummy_row = []
        dummy_bias = []
        #number of weights = neurons(i)*neurons(i+1)
        for j in range(0,neurons_per_layer[i+1]): 
            weight_set = []
            dummy_set = []
            #weights[0][0] = [weights(first input neuron) to first neuron of next layer]
            for k in range(0,neurons_per_layer[i]): 
                weight_set.append(random.uniform(0,1))
                dummy_set.append(0)
            weight_row.append(weight_set)
            dummy_row.append(dummy_set)
            bias_row.append(+1.0)
            dummy_bias.append(0)
        bias_matrix.append(bias_row)
        weight_matrix.append(weight_row)
        change_in_weight_matrix.append(dummy_row)
        change_in_bias_matrix.append(dummy_bias)
        
    ##training
    maxSamples = train_set_length
    
    number_of_epochs = 0
    
    while number_of_epochs < max_iterations:
        test = 0 ##sample number
        final_output = []
        while test < maxSamples:
            ## initialising the output,derivative,error correction matrices
            output_matrix = []
            derivative_matrix = []
            error_correction_matrix = []

            #first row contains second layer outputs,last row contains output layer outputs
            for i in range(0,number_of_layers-1): 
                dummy_row = []
                for j in range(0,neurons_per_layer[i+1]):
                    dummy_row.append(0)
                output_matrix.append(dummy_row)

            for i in range(0,number_of_layers-1):
                dummy_row = []
                for j in range(0,neurons_per_layer[i+1]):
                    dummy_row.append(0)
                derivative_matrix.append(dummy_row)

            for i in range(0,number_of_layers-1): 
                dummy_row = []
                for j in range(0,neurons_per_layer[i+1]):
                    dummy_row.append(0)
                error_correction_matrix.append(dummy_row)

            ##filling the output and derivative matrices
            for i in range(0,len(weight_matrix)):
                for j in range(0,len(weight_matrix[i])):
                    output = 0
                    for k in range (0,len(weight_matrix[i][j])):
                        if i == 0:
                            output += weight_matrix[i][j][k]*input_matrix[test][k]
                        else:
                            output += weight_matrix[i][j][k]*output_matrix[i-1][k]
                    output += bias_matrix[i][j]
                    output_matrix[i][j] = activation_function(output)
                    derivative_matrix[i][j] = activation_func_derivate(output_matrix[i][j])

            ##filling the error correction matrices
            i = len(error_correction_matrix)-1

            for j in range(0,len(error_correction_matrix[i])):
                error_correction_matrix[i][j] = (target_list[test]-output_matrix[i][j])*derivative_matrix[i][j]

            i -= 1

            while i>=0:
                for j in range(0,len(error_correction_matrix[i])):
                    for k in range(0,len(error_correction_matrix[i+1])):
                        error_correction_matrix[i][j] += (error_correction_matrix[i+1][k]*weight_matrix[i+1][k][j])
                    error_correction_matrix[i][j] *= derivative_matrix[i][j]
                i -= 1

            ##filling the change in weight and change in bias matrices

            for i in range(0,len(change_in_weight_matrix)):
                for j in range(0,len(change_in_weight_matrix[i])):
                    change_in_bias_matrix[i][j] = learning_rate*error_correction_matrix[i][j]
                    for k in range(0,len(change_in_weight_matrix[i][j])):
                        if i!=0:
                            change_in_weight_matrix[i][j][k] = learning_rate*error_correction_matrix[i][j]*output_matrix[i-1][k]
                        else:
                            change_in_weight_matrix[i][j][k] = learning_rate*error_correction_matrix[i][j]*input_matrix[test][k]


            ##changing weights
            for i in range(0,number_of_layers-1): 
                for j in range(0,neurons_per_layer[i+1]): 
                    for k in range(0,neurons_per_layer[i]):
                        weight_matrix[i][j][k] += change_in_weight_matrix[i][j][k]
                bias_matrix[i][j] += change_in_bias_matrix[i][j]
            if number_of_epochs == max_iterations-1:
                final_output.append([round(output_matrix[-1][-1],2),target_list[test]])
            test += 1
        number_of_epochs += 1
        
    mse_train = mean_square_error(pair_list=final_output)
    sse_train = sum_squared_error(pair_list=final_output)
    abs_error_train = mean_absolute_error(pair_list=final_output)
    
    ##extracting data for testing
    test_input = [row[:-1] for row in test_set]
    test_target = [row[-1] for row in test_set]
    index = 0
    test_output = []

    for row in test_input:
        output_matrix = []

        #first row contains second layer outputs,last row contains output layer outputs
        for i in range(0,number_of_layers-1): 
            dummy_row = []
            for j in range(0,neurons_per_layer[i+1]):
                dummy_row.append(0)
            output_matrix.append(dummy_row)
        
        test_output.append([test_target[index],round(test_case(inputs=row,weights=weight_matrix,bias=bias_matrix,output_matrix=output_matrix),2)])
        index += 1
    
    mse_test = mean_square_error(pair_list=test_output)
    sse_test = sum_squared_error(pair_list=test_output)
    abs_error_test = mean_absolute_error(pair_list=test_output)
    
    return mse_train,sse_train,abs_error_train,mse_test,sse_test,abs_error_test

In [43]:
mse1,sse1,abs1,mse2,sse2,abs2 = back_propagation_single_output(dataset_path='fourvars.csv',max_iterations=10000,neurons_per_hidden_layer=[4,3,2])

print('Mean Square Error(Train) ',mse1)
print('Mean Square Error(Test) ',mse2)
print('Sum Squared Error(Train) ',sse1)
print('Sum Squared Error(Test) ',sse2)
print('Mean Absolute Error(Train) ',abs1)
print('Mean Absolute Error(Test) ',abs2)

[[0.12, 0.56, 0.41, 0.59], [0.6, 0.88, 0.32, 0.3]]
Mean Square Error(Train)  0.0
Mean Square Error(Test)  0.032650000000000005
Sum Squared Error(Train)  0.0
Sum Squared Error(Test)  0.06530000000000001
Mean Absolute Error(Train)  0.0
Mean Absolute Error(Test)  0.35000000000000003
